## Experiments of different models

### Get Experimental Data

In [ ]:
import numpy as np
import os
import Preprocess
import platform

file_path = os.path.abspath(os.path.dirname(os.path.dirname(os.path.abspath(''))))

if platform.system().lower() == 'windows':
    print("windows")
    dataset_X = np.load(file_path + "\Classification_smooth\data\dataset_X.npy")
    dataset_Y = np.load(file_path + "\Classification_smooth\data\dataset_Y.npy")
elif platform.system().lower() == 'linux':
    print("linux")    
    dataset_X = np.load(file_path + "/Classification_smooth/data/dataset_X.npy")
    dataset_Y = np.load(file_path + "/Classification_smooth/data/dataset_Y.npy")



data_X = []
data_Y = []

num = 24

for i in range(0, len(dataset_X)-num, 24):
    data_X.append(dataset_X[i:i+num])
    data_Y.append(dataset_Y[i:i+num])
    
data_X = np.array(data_X)
data_Y = np.array(data_Y)

train_X = data_X[:1200]
train_Y = data_Y[:1200]
test_X = data_X[1200:]
test_Y = data_Y[1200:]

Preprocess.save_as_npy(train_X, train_Y, test_X, test_Y)



In [ ]:
import numpy as np

file_path = os.path.abspath(os.path.dirname(os.path.dirname(os.path.abspath(''))))


if platform.system().lower() == 'windows':
    print("windows")
    train_X = np.load(file_path + "\Classification_smooth\data\\train_X.npy")
    train_Y = np.load(file_path + "\Classification_smooth\data\\train_Y.npy")
    test_X = np.load(file_path + "\Classification_smooth\data\\test_X.npy")
    test_Y = np.load(file_path + "\Classification_smooth\data\\test_Y.npy")
elif platform.system().lower() == 'linux':
    print("linux")    
train_X = np.load(file_path + "/Classification_smooth/data/train_X.npy")
train_Y = np.load(file_path + "/Classification_smooth/data/train_Y.npy")
test_X = np.load(file_path + "/Classification_smooth/data/test_X.npy")
test_Y = np.load(file_path + "/Classification_smooth/data/test_Y.npy")



### Construct dataset

In [ ]:
from models.customed_dataset import customed_dataset
import numpy as np
import torch
import os

file_path = os.path.abspath(os.path.dirname(os.path.dirname(os.path.abspath(''))))

# dataset_X = np.load(file_path + "\Task_2\data\\dataset_X.npy")
# dataset_Y = np.load(file_path + "\Task_2\data\\dataset_Y.npy")

# dataset = customed_dataset(dataset_X, dataset_Y)

# train_size = int(0.8 * len(dataset))
# test_size = len(dataset) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_dataset = customed_dataset(train_X, train_Y)
test_dataset = customed_dataset(test_X, test_Y)

training_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
validation_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=0)


### LSTM model

In [ ]:
from models import lstm

from datetime import datetime
import torch

model = lstm.lstm_model().cuda()
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
epoch_number = 0

EPOCHS = 2000

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = lstm.train_one_epoch(model, epoch_number, training_loader, optimizer, loss_fn)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(validation_loader):
        vinputs, vlabels = vdata
        vinputs = vinputs.float().cuda()
        vlabels = vlabels.float().cuda()
        voutputs = model(vinputs)
        voutputs = torch.squeeze(voutputs)
        #print(voutputs.size())
        #print(vlabels.size())
        vloss = loss_fn(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))


    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'models/LSTM_model'
        torch.save(model.state_dict(), model_path)

    epoch_number += 1


### GRU

In [ ]:
from models import gru

from datetime import datetime
import torch

model = gru.gru_model().cuda()
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
epoch_number = 0

EPOCHS = 2000

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = gru.train_one_epoch(model, epoch_number, training_loader, optimizer, loss_fn)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    h_0 = torch.zeros(1, 4, 32).cuda()
    for i, vdata in enumerate(validation_loader):
        vinputs, vlabels = vdata
        vinputs = vinputs.float().cuda()
        vlabels = vlabels.float().cuda()
        if vinputs.size()[0] != 4:
            continue    
        voutputs = model(vinputs)
        voutputs = torch.squeeze(voutputs)
        #print(voutputs.size())
        #print(vlabels.size())
        vloss = loss_fn(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))



    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'models/GRU_model'
        torch.save(model.state_dict(), model_path)

    epoch_number += 1


### Test of Models

#### Load Models

In [ ]:
import torch

from models import lstm
from models import gru
from models import transformer



### lstm
model_lstm = lstm.lstm_model()
model_lstm.load_state_dict(torch.load('models/LSTM_model'))
model_lstm.eval()

### gru
model_gru = gru.gru_model()
model_gru.load_state_dict(torch.load('models/GRU_model'))
model_gru.eval()

#### lstm test

In [ ]:
import torch
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

lstm_predictions = []
lstm_targets = []

validation_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

for batch_idx, (data, target) in enumerate(validation_loader):
    lstm_predictions.append(model_lstm.forward(data.float()).detach().numpy().tolist())
    lstm_targets.append(target.numpy().tolist())
lstm_predictions = torch.squeeze(torch.tensor(lstm_predictions)).reshape(-1).numpy()
lstm_targets = torch.squeeze(torch.tensor(lstm_targets)).reshape(-1).numpy()

for i in range(0, len(lstm_predictions)):
    if lstm_predictions[i] <= 35:
        lstm_predictions[i] = 0
    elif lstm_predictions[i] <= 150:
        lstm_predictions[i] = 1
    else:
        lstm_predictions[i] = 2
    if lstm_targets[i] <= 35:
        lstm_targets[i] = 0
    elif lstm_targets[i] <= 150:
        lstm_targets[i] = 1
    else:
        lstm_targets[i] = 2
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
print(accuracy_score(lstm_targets, lstm_predictions))
print(f1_score(lstm_targets, lstm_predictions, average='macro'))

### gru test

In [ ]:
import torch
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

model_gru = model_gru.cuda()

gru_predictions = []
gru_targets = []

validation_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=0)

for batch_idx, (data, target) in enumerate(validation_loader):
    if len(data) != 4:
        continue
    gru_predictions.append(model_gru.forward(data.float().cuda()).cpu().detach().numpy().tolist())
    gru_targets.append(target.numpy().tolist())
gru_predictions = torch.squeeze(torch.tensor(gru_predictions)).reshape(-1).numpy()
gru_targets = torch.squeeze(torch.tensor(gru_targets)).reshape(-1).numpy()

for i in range(0, len(gru_predictions)):
    if gru_predictions[i] <= 35:
        gru_predictions[i] = 0
    elif gru_predictions[i] <= 150:
        gru_predictions[i] = 1
    else:
        gru_predictions[i] = 2
    if gru_targets[i] <= 35:
        gru_targets[i] = 0
    elif gru_targets[i] <= 150:
        gru_targets[i] = 1
    else:
        gru_targets[i] = 2

print(accuracy_score(gru_targets, gru_predictions))
print(f1_score(gru_targets, gru_predictions,average='macro'))